In [1]:
# Disable tensorflow warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# 0 = all messages are logged (default behavior)
# 1 = INFO messages are not printed
# 2 = INFO and WARNING messages are not printed
# 3 = INFO, WARNING, and ERROR messages are not printed

In [2]:
from model import MukkeBudeTransformer
from mapping import MusicMapping
import utils
import music21 as m21
import tensorflow as tf
import numpy as np
from pathlib import Path

# Check if GPU is found
print(tf.config.list_physical_devices('GPU'))

[]


In [3]:
# Create mappings
mapping = MusicMapping.create()
mapping.save("mapping.txt")

In [4]:
# Load songs
paths = m21.corpus.getComposer('bach')
# paths = paths[:100]
print(f"Found {len(paths)} songs in corpus.")

encoded_songs = []
for path in paths:
    song = utils.read_single_from_corpus(path)
    encoded_song = utils.to_polyphonic_encoding(song, mapping)
    encoded_songs.append(mapping.textify(encoded_song))

print(f"Songs encoded: {len(encoded_songs)}")

Found 433 songs in corpus.


d:\Projekte\Python\Tensorflow\Komposition-eines-Musikstuecks-mittels-Neuronaler-Netze\mukkeBude\utils.py:137: StreamIteratorInefficientWarning: highestTime is not defined on StreamIterators. Call .stream() first for efficiency
  song.flat.getElementsByClass("Note").highestTime,
d:\Projekte\Python\Tensorflow\Komposition-eines-Musikstuecks-mittels-Neuronaler-Netze\mukkeBude\utils.py:138: StreamIteratorInefficientWarning: highestTime is not defined on StreamIterators. Call .stream() first for efficiency
  song.flat.getElementsByClass("Chord").highestTime,
d:\Projekte\Python\Tensorflow\Komposition-eines-Musikstuecks-mittels-Neuronaler-Netze\mukkeBude\utils.py:137: StreamIteratorInefficientWarning: highestTime is not defined on StreamIterators. Call .stream() first for efficiency
  song.flat.getElementsByClass("Note").highestTime,
d:\Projekte\Python\Tensorflow\Komposition-eines-Musikstuecks-mittels-Neuronaler-Netze\mukkeBude\utils.py:138: StreamIteratorInefficientWarning: highestTime is not

Songs encoded: 433


In [5]:
# Create dataset
utils.create_train_data(encoded_songs, "raw_train_ds.txt")
print("Dataset created")

Dataset created


In [6]:
# Train model
model = MukkeBudeTransformer(mapping)
print(model)
model.train("raw_train_ds.txt", min_training_seq_len=32, epochs=30)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 token_and_position_embeddin  (None, None, 256)        652288    
 g (TokenAndPositionEmbeddin                                     
 g)                                                              
                                                                 
 transformer_decoder (Transf  (None, None, 256)        394749    
 ormerDecoder)                                                   
                                                                 
 transformer_decoder_1 (Tran  (None, None, 256)        394749    
 sformerDecoder)                                                 
                                                                 
 transformer_decoder_2 (Tran  (None, None, 256)        394749

In [12]:
# Create song
generated_song = model.generate("xxbos n67 d4 n62 d4 n58 d4 n43 d4 xxsep d4 n67 d4 n62 d4 n58 d4 n55 d4 xxsep d4 n69 d4 n62 d4 n57 d4 n54 d4 xxsep", max_length=2048, probability=0.8)
new_song_str = generated_song.numpy().decode("utf-8")
print(f"CLEAN SONG: {new_song_str}\n\n")

# THIS DEPENDS ON THE OUTPUT - NEED TO FIX THE OUTPUT
# new_song_str = new_song_str[15:]
# new_song_str = "xxbos xxpad " + new_song_str
# new_song_str = new_song_str.replace("   ", " ")

print(new_song_str)

prompt_tokens: [ 2 23 18 20 18 38 18 51 18  4 18 23 18 20 18 38 18 28 18  4 18 22 18 20
 18 25 18 37 18  4]
CLEAN SONG: xxbos n67 d4 n62 d4 n58 d4 n43 d4 xxsep d4 n67 d4 n62 d4 n58 d4 n55 d4 xxsep d4 n69 d4 n62 d4 n57 d4 n54 d4 xxsep d4 n62 d4 xxsep d4 n67 d4 n58 d4 xxsep d4 n74 d4 n67 d4 n62 d4 n59 d4 xxsep d4 n69 d4 n62 d2 n54 d2 xxsep d2 n55 d4 n67 d4 n62 d2 n57 d4 n54 d4 xxsep d2 n67 d4 n59 d2 n55 d4 xxsep d2 n60 d2 xxsep d2 n62 d4 n57 d2 xxsep d2 n53 d2 xxsep d2 n67 d4 n59 d4 n55 d4 xxsep d4 n69 d4 n66 d4 n62 d4 n59 d4 n50 d4 xxsep d4 n67 d4 n62 d4 n54 d4 xxsep d4 n71 d4 n67 d4 n62 d4 n55 d4 n62 d4 n62 d4 n67 d4 n64 d4 n59 d4 n62 d4 n62 d4 n55 d4 n62 d4 n64 d4 n62 d4 n62 d4 n55 d4 n59 d4 n55 d4 n55 d4 n62 d4 n59 d4 n55 d4 n55 d4 n55 d4 n62 d4 n62 d4 n59 d4 n62 d4 n67 d4 n55 d4 n59 d4 n55 d4 n55 d4 n55 d4 n62 d4 n59 d4 n55 d4 n62 d4 n59 d4 n62 d4 n55 d4 n62 d4 n62 d4 n62 d4 n62 d4 n55 d4 n59 d4 n55 d4 n59 d4 n59 d4 n64 d4 xxsep d4 n55 d4 n55 d4 n55 d4 n43 d4 n62 d4 n52 d4 n55 d4 n6

In [ ]:
# Convert to music21
new_song_ints = mapping.numericalize(new_song_str.split(" "))
new_song_ints = np.array(new_song_ints)

print(new_song_ints)
print(mapping.textify(new_song_ints))

new_song = utils.from_polyphonic_encoding(new_song_ints, mapping, bpm=100)
path = Path("generated_song_bach.mid")
utils.write_midi(new_song, path)
utils.write_musicxml(new_song, "generated_song_bach.musicxml")

[  0  71 136 ... 136  59 136]
xxbos n67 d4 n62 d4 n58 d4 n43 d4 xxsep d4 n67 d4 n62 d4 n58 d4 n55 d4 xxsep d4 n69 d4 n62 d4 n57 d4 n54 d4 xxsep d4 n62 d4 xxsep d4 n67 d4 n58 d4 xxsep d4 n74 d4 n67 d4 n62 d4 n59 d4 xxsep d4 n69 d4 n62 d2 n54 d2 xxsep d2 n55 d4 n67 d4 n62 d2 n57 d4 n54 d4 xxsep d2 n67 d4 n59 d2 n55 d4 xxsep d2 n60 d2 xxsep d2 n62 d4 n57 d2 xxsep d2 n53 d2 xxsep d2 n67 d4 n59 d4 n55 d4 xxsep d4 n69 d4 n66 d4 n62 d4 n59 d4 n50 d4 xxsep d4 n67 d4 n62 d4 n54 d4 xxsep d4 n71 d4 n67 d4 n62 d4 n55 d4 n62 d4 n62 d4 n67 d4 n64 d4 n59 d4 n62 d4 n62 d4 n55 d4 n62 d4 n64 d4 n62 d4 n62 d4 n55 d4 n59 d4 n55 d4 n55 d4 n62 d4 n59 d4 n55 d4 n55 d4 n55 d4 n62 d4 n62 d4 n59 d4 n62 d4 n67 d4 n55 d4 n59 d4 n55 d4 n55 d4 n55 d4 n62 d4 n59 d4 n55 d4 n62 d4 n59 d4 n62 d4 n55 d4 n62 d4 n62 d4 n62 d4 n62 d4 n55 d4 n59 d4 n55 d4 n59 d4 n59 d4 n64 d4 xxsep d4 n55 d4 n55 d4 n55 d4 n43 d4 n62 d4 n52 d4 n55 d4 n62 d4 xxsep d4 n59 d4 n55 d4 n59 d4 n55 d4 n55 d4 n62 d4 n55 d4 n62 d4 n55 d4 n55 d4 n62 d4

In [9]:
# Open song in MuseScore (Linux if installed)
!musescore4portable {path.absolute()}

Der Befehl "musescore4portable" ist entweder falsch geschrieben oder
konnte nicht gefunden werden.
